In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from common import *
import pandas as pd
#from mpl_toolkits.mplot3d import Axes3D
import sklearn.linear_model as lm

print(f"Root directory: {os.getcwd()}")

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
    "font.size": 10,
})

#%matplotlib inline 

### Global variables 

#  Franke function 3D preview
First we plot a 3D plot of the franke function.
The plot is based on the provided code in the assignmentext for plotting the franke function 

In [ ]:
# Preview plot of the franke function
#%matplotlib
np.random.seed(4155)
y = x = np.arange(0, 1, 0.05)
x, y = np.meshgrid(x,y)
z = FrankeFunction(x, y)
noise_mesh = 0.05 * np.random.randn(z.shape[0], z.shape[1]) # Stochastic noise
z_noisy = z + noise_mesh

fig = plt.figure()
# Ploting frankefunction without noise
ax1 = fig.add_subplot(1,2,1, projection='3d') # Are :)steike
ax1.title.set_text("Plot of the Franke Function")
ax1.view_init(elev=30., azim=-25.0)
ax1.set_xlabel("x"); ax1.set_ylabel("y"); ax1.set_zlabel("z")
surf1 = ax1.plot_surface(x,y,z, cmap=cm.coolwarm, linewidth = 0, antialiased=False)
# Customize the z axis.
ax1.set_zlim(-0.10, 1.40)
ax1.zaxis.set_major_locator(LinearLocator(10))
ax1.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# Ploting frankefunction with noise
ax2 = fig.add_subplot(1,2,2, projection='3d')
ax2.title.set_text("Plot of the Franke Function\n(noise added)")
ax2.view_init(elev=30., azim=-25.0)
ax2.set_xlabel("x"); ax2.set_ylabel("y"); ax2.set_zlabel("z")
surf2 = ax2.plot_surface(x,y,z_noisy, cmap=cm.coolwarm, linewidth = 0, antialiased=False)
# Customize the z axis.
ax2.set_zlim(-0.10, 1.40)
ax2.zaxis.set_major_locator(LinearLocator(10))
ax2.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))
plt.savefig(f"{REPORT_FIGURES}franke_function_preview.pdf")
plt.show()

# 1 - Ordinary Least Squeares (OLS)

## 1.1 Data
Defining and creating the data


In [ ]:
np.random.seed(4155)
n = 100 # The number of point in the franke function
x = np.random.rand(n)
y = np.random.rand(n)
x, y = np.meshgrid(x,y)
z = FrankeFunction(x, y) 
noise = 0.5 * np.random.randn(n,n) # Stochastic noise
#noise = 0.7 * np.random.randn(n) # Stochastic noise
z_noise = z + noise # separate Franke with noise
#z += noise # Adding stochastic noise

## 1.2 Plot of fit for all degrees before evaluation
 We plot the fit up to degree 6 to get an intuition on the curvature of the fitted models

In [ ]:
fig = plt.figure(figsize=(8,8))
degrees = 6
z_train_OLS = pd.DataFrame()
z_hat_train_OLS = pd.DataFrame()
z_test_OLS = pd.DataFrame()
z_hat_test_OLS = pd.DataFrame()

# TODO: Must fix so that training and test data are used. 
# Must evalute model using MSE from traning and test
for degree in range(1, degrees + 1):
    X = create_X(x, y, degree) # Design MatrixS
    model = OLS() # The model
    #model.fit(X, z) # Fitting the model
    model.fit(X, z.ravel()) # Fitting the model
    z_hat = model.predict(X) # predict on train data
    
    # Plot
    ax = fig.add_subplot(3,2, degree, projection='3d')
    ax.view_init(elev=30., azim=-25.0)
    ax.title.set_text(f"OLS/Linear fit of degree{degree}")
    ax.set_xlabel("x"); ax.set_ylabel("y"); ax.set_zlabel("z")
    ax.scatter3D(y, x, z_hat, c=z_hat ,marker = '.', cmap=cm.coolwarm)
fig.suptitle("OLS fit to the Franke Function")
plt.tight_layout()
plt.savefig(f"{REPORT_FIGURES}franke_function_OLS_fit.pdf")
plt.show()

## 1.3 - Finding degree/model complexity for the optimal OLS fit
Approximate the franke function using ordinary least squares
We estimate the franke functinon using polynomials up to 6th degree. We than look at the MSE scores to look for overfitting. We use the MSE score values from the test data to determine overfit together with the curvature of the evaluation plot

### Confidence intervall 
$$CI_{0.95}(\hat\beta_i) = [\hat\beta_i-1.96 SE(\hat\beta_i), \hat\beta_i+1.96 SE(\hat\beta_i)] =\hat\beta_i \pm 1.96\hat \sigma(\hat\beta_i)$$ 
In order to estimate the variance of the $i$-th beta values: $$\sigma^2 (\beta_i ) = \sigma^2 [(X^{T} X)^{-1}]_{ii}$$
However, $\sigma$ is unkown and can be generaly estimated as followed:
$$\hat\sigma^2 = \frac{\sum_{i=0}^{N-1}(y_i - \hat y_i)^2}{N}$$
For simplification purposes, we N instead of N-p-1 in the denominator.<br>
To get the variance estimate of each $\beta$ component one must calculate the variance with respect to the diagonal elements of $(X^TX)^{-1}$ Estimated standard error is the square root of $\hat\sigma^2$, where the estimate for variance $\hat\sigma^2$ is:
$$\hat\sigma^2 = \frac{\sum_{i=0}^{N-1}(y_i - \hat y_i)^2}{N}(X^TX)^{-1}$$
Where y is the true value, and $\hat y$ being the predicted value. <br>
The variance estimate of each $\hat\beta$ estimate can be written as:
$$\hat\sigma_{\hat\beta_i}^2 = \hat\sigma^2(X^TX)_{i,i}^{-1}$$



In [ ]:
degrees = 5

# Setting up dataframes for the observed values
z_train_OLS = pd.DataFrame()
z_test_OLS = pd.DataFrame()

# Setting up the dataframes for our computed values
z_hat_train_OLS = pd.DataFrame()
z_hat_test_OLS = pd.DataFrame()

# Setting up the dataframes for our SVD computed values
z_hat_train_SVD = pd.DataFrame()
z_hat_test_SVD = pd.DataFrame()

# Setting up dataframes for sklearn computed values
z_hat_train_SK = pd.DataFrame()
z_hat_test_SK = pd.DataFrame()

coeffs_df = pd.DataFrame()

for degree in range(1, degrees+1):
    X = create_X(x, y, degree) # Design Matrix
    
    # Scaling data and splitting it into training and test sets
    #X_train, X_test, z_train, z_test = prepare_data(X, z, test_size=0.2, shuffle=True, scale_X=False, scale_t=False)
    X_train, X_test, z_train, z_test = prepare_data(X, z.ravel(), test_size=0.2, shuffle=True, scale_X=False, scale_t=False, random_state=4155)
    
    # Model construction, fitting, and predictions using matrix inversion
    model = OLS() # The model
    z_hat_train = model.fit(X_train, z_train, SVDfit = False) # Fitting the model and predict on training data
    z_hat_test = model.predict(X_test) # predict on test data

    # Model construction, fitting and predictions using SVD inversion
    model_svd = OLS()
    z_hat_train_svd = model_svd.fit(X_train, z_train)
    z_hat_test_svd = model.predict(X_test)

    # Model construction, fitting and predictions using sklearn
    model_sk = lm.LinearRegression()
    model_sk.fit(X_train, z_train)
    z_hat_train_sk = model_sk.predict(X_train)
    z_hat_test_sk = model_sk.predict(X_test)
    
    # Evaluatation metrics OLS, SVD, SK
    MSE_score_train = MSE(z_train, z_hat_train)
    R2_score_train = R2(z_train, z_hat_train)
    MSE_score_test = MSE(z_test, z_hat_test)
    R2_score_test = R2(z_test, z_hat_test)

    MSE_score_train_svd = MSE(z_train, z_hat_train_svd)
    R2_score_train_svd = R2(z_train, z_hat_train_svd)
    MSE_score_test_svd = MSE(z_test, z_hat_test_svd)
    R2_score_test_svd = R2(z_test, z_hat_test_svd)

    MSE_score_train_sk = MSE(z_train, z_hat_train_sk)
    R2_score_train_sk = R2(z_train, z_hat_train_sk)
    MSE_score_test_sk = MSE(z_test, z_hat_test_sk)
    R2_score_test_sk = R2(z_test, z_hat_test_sk)
    
    # Estimated standard error for the beta coefficients
    SE_betas = model.SE
    SE_betas_svd = model_svd.SE

    #N, P = X_train.shape
    #var_hat = (1/(N-P-1)) * np.sum((z_train - z_hat_train)**2)
    var_hat = (1/X_train.shape[0]) * np.sum((z_train - z_hat_train)**2) # Estimated variance
    #invXTX_diag = np.diag(np.linalg.inv(X_train.T @ X_train)) 
    #SE_betas = np.sqrt(var_hat * invXTX_diag) # Standard Error

    # Calculating 95% confidence intervall OLS, SVD
    betas = model.get_all_betas
    CI_lower_all_betas = betas - (1.96 * SE_betas)
    CI_upper_all_betas = betas + (1.96 * SE_betas)
    CL = np.zeros((CI_upper_all_betas.shape[0],2))
    CL[:,0] = CI_lower_all_betas
    CL[:,1] = CI_upper_all_betas

    betas_svd = model.get_all_betas

        
    # Printing results
    print(f"Degree: {degree}")
    print(f"Train data - Mean Square Error: {MSE_score_train}")
    print(f"Train data - R2 score: {R2_score_train}")
    print(f"Test data - Mean Square Error: {MSE_score_test}")
    print(f"Test data - R2 score: {R2_score_test}")
    print(f"Estimated variance: {var_hat}")
    print(f"Beta - values{betas}")
    print(f"Beta - Std Errors: {SE_betas}")
    print(f"Beta - Confidence Interval (CI):\n{CL}\n")

    # Constructing dataframe for beta coefficients
    degs = np.zeros(betas.shape[0]); degs.fill(degree)
    df = pd.DataFrame.from_dict({"degree" :degs,
                                 "coeff name": [f"b{i}" for i in range(1,betas.shape[0]+1)],
                                 "coeff value": np.round(betas, decimals=4),
                                 "Std Error": np.round(SE_betas, decimals=4),
                                 "CI lower":np.round(CI_lower_all_betas, decimals=4), 
                                 "CI_upper":np.round(CI_upper_all_betas, decimals=4)},
                                 orient='index').T
    coeffs_df = pd.concat([coeffs_df,df], axis=0)
        
    # Filling up dataframes for train and test evaluation
    z_train_OLS[degree] = z_train.flatten() 
    z_hat_train_OLS[degree] = z_hat_train.flatten()
    z_test_OLS[degree] = z_test.flatten()
    z_hat_test_OLS[degree] = z_hat_test.flatten()

    z_hat_train_SVD[degree] = z_hat_train_svd.flatten()
    z_hat_test_SVD[degree] = z_hat_test_svd.flatten()

    z_hat_train_SK[degree] = z_hat_train_sk.flatten()
    z_hat_test_SK[degree] = z_hat_test_sk.flatten()


# MSE calculations for all lambda values
mse_scores_train = ((z_train_OLS - z_hat_train_OLS) ** 2).mean()
mse_scores_test = ((z_test_OLS - z_hat_test_OLS) ** 2).mean()

mse_scores_train_svd = ((z_train_OLS - z_hat_train_SVD) ** 2).mean()
mse_scores_test_svd = ((z_test_OLS - z_hat_test_SVD) ** 2).mean()

mse_scores_train_sk = ((z_train_OLS - z_hat_train_SK) ** 2).mean()
mse_scores_test_sk = ((z_test_OLS - z_hat_test_SK) ** 2).mean()

# R2 calculations for all lambda values
R2_scores_train = 1 - ((z_train_OLS - z_hat_train_OLS) ** 2).sum() / ((z_train_OLS - z_train_OLS.mean())**2).sum() 
R2_scores_test = 1 - ((z_test_OLS - z_hat_test_OLS) ** 2).sum() / ((z_test_OLS - z_test_OLS.mean())**2).sum()

In [ ]:
# Plots
plt.figure(figsize=(12,8))
plt.plot(np.arange(1,degrees+1), mse_scores_train, c="c", label="Training data")
plt.plot(np.arange(1,degrees+1), mse_scores_test, c="m", label="Test data")
plt.plot(np.arange(1,degrees+1), mse_scores_train_svd, "b--", label="Training data SVD")
plt.plot(np.arange(1,degrees+1), mse_scores_test_svd, "g--", label="Test data SVD")
plt.plot(np.arange(1,degrees+1), mse_scores_train_sk, "r--", label="Training data sklearn")
plt.plot(np.arange(1,degrees+1), mse_scores_test_sk, "y--", label="Test data sklearn")
plt.xlabel("Model complexity / Polynomial Degree")
plt.ylabel("Prediction Error - MSE")
plt.title("Training evaluation on OLS regression fit")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(f"{REPORT_FIGURES}franke_function_OLS_evaluate_fit.pdf")
plt.show()

## 1.4 Analysis of plots and training metrics
Do the analysis.......

Conclusion:<br>
Based on the analysis, we conclude that a model complexity of degree 5 yields the most optimal fit.

## 1.5 plot of the model using the most optimal parameters

In [ ]:
optimal_degree = 5
X = create_X(x, y, optimal_degree) # Design Matrix
model = OLS() # The model
model.fit(X, z.ravel()) # Fitting the model
z_hat = model.predict(X) # predict on train data

# Plot
fig = plt.figure()#figsize=(8,8))
ax = plt.axes(projection='3d')
ax.title.set_text(f"OLS regression fit to the Franke Function\noptimal degree {optimal_degree},")
#ax.view_init(elev=5., azim=85.0)
ax.view_init(elev=30., azim=-25.0)
ax.set_xlabel("x"); ax.set_ylabel("y"); ax.set_zlabel("z")
ax.scatter3D(y, x, z_hat, c=z_hat ,marker = '.', cmap=cm.coolwarm)
plt.savefig(f"{REPORT_FIGURES}franke_function_OLS_best_fit.pdf")
plt.show()

In [ ]:
optimal_degree_coeffs = coeffs_df[coeffs_df["degree"] == optimal_degree]
display(optimal_degree_coeffs)
optimal_degree_coeffs.to_csv(f"{REPORT_DATA}EX1_coeffs_optimal_degree.csv")

Repeat of the previous exercise, this time with scaled data to study the difference for OLS with and without scaled data
(Hint: there should be no difference in MSE for OLS given scaled data)

In [ ]:
# Setting up dataframes for the observed values
z_train_OLS = pd.DataFrame()
z_test_OLS = pd.DataFrame()

# Setting up the dataframes for our computed values
z_hat_train_OLS = pd.DataFrame()
z_hat_test_OLS = pd.DataFrame()

# Setting up the dataframes for our SVD computed values
z_hat_train_SVD = pd.DataFrame()
z_hat_test_SVD = pd.DataFrame()

# Setting up dataframes for sklearn computed values
z_hat_train_sk = pd.DataFrame()
z_hat_test_sk = pd.DataFrame()

coeffs_df = pd.DataFrame()

for degree in range(1, degrees+1):
    X = create_X(x, y, degree)[:,1:] # Design Matrix
    
    # Scaling data and splitting it into training and test sets
    #X_train, X_test, z_train, z_test = prepare_data(X, z, test_size=0.2, shuffle=True, scale_X=False, scale_t=False)
    X_train, X_test, z_train, z_test = prepare_data(X, z.ravel(), test_size=0.2, shuffle=True, scale_X=True, scale_t=True, zero_center = True, random_state=4155)
    
    # Model construction, fitting, and predictions using matrix inversion
    model = OLS() # The model
    z_hat_train = model.fit(X_train, z_train, SVDfit = False) # Fitting the model and predict on training data
    z_hat_test = model.predict(X_test) # predict on test data

    # Model construction, fitting and predictions using SVD inversion
    model_svd = OLS()
    z_hat_train_svd = model_svd.fit(X_train, z_train)
    z_hat_test_svd = model.predict(X_test)

    # Model construction, fitting and predictions using sklearn
    model_sk = lm.LinearRegression(fit_intercept=False)
    model_sk.fit(X_train, z_train)
    z_hat_train_sk = model_sk.predict(X_train)
    z_hat_test_sk = model_sk.predict(X_test)
    
    # Evaluatation metrics OLS, SVD, SK
    MSE_score_train = MSE(z_train, z_hat_train)
    R2_score_train = R2(z_train, z_hat_train)
    MSE_score_test = MSE(z_test, z_hat_test)
    R2_score_test = R2(z_test, z_hat_test)

    MSE_score_train_svd = MSE(z_train, z_hat_train_svd)
    R2_score_train_svd = R2(z_train, z_hat_train_svd)
    MSE_score_test_svd = MSE(z_test, z_hat_test_svd)
    R2_score_test_svd = R2(z_test, z_hat_test_svd)

    MSE_score_train_sk = MSE(z_train, z_hat_train_sk)
    R2_score_train_sk = R2(z_train, z_hat_train_sk)
    MSE_score_test_sk = MSE(z_test, z_hat_test_sk)
    R2_score_test_sk = R2(z_test, z_hat_test_sk)
    
    # Estimated standard error for the beta coefficients
    SE_betas = model.SE
    SE_betas_svd = model_svd.SE

    #N, P = X_train.shape
    #var_hat = (1/(N-P-1)) * np.sum((z_train - z_hat_train)**2)
    var_hat = (1/X_train.shape[0]) * np.sum((z_train - z_hat_train)**2) # Estimated variance
    #invXTX_diag = np.diag(np.linalg.inv(X_train.T @ X_train)) 
    #SE_betas = np.sqrt(var_hat * invXTX_diag) # Standard Error

    # Calculating 95% confidence intervall OLS, SVD
    betas = model.get_all_betas
    CI_lower_all_betas = betas - (1.96 * SE_betas)
    CI_upper_all_betas = betas + (1.96 * SE_betas)
    CL = np.zeros((CI_upper_all_betas.shape[0],2))
    CL[:,0] = CI_lower_all_betas
    CL[:,1] = CI_upper_all_betas

    betas_svd = model.get_all_betas

        
    # Printing results
    print(f"Degree: {degree}")
    print(f"Train data - Mean Square Error: {MSE_score_train}")
    print(f"Train data - R2 score: {R2_score_train}")
    print(f"Test data - Mean Square Error: {MSE_score_test}")
    print(f"Test data - R2 score: {R2_score_test}")
    print(f"Estimated variance: {var_hat}")
    print(f"Beta - values{betas}")
    print(f"Beta - Std Errors: {SE_betas}")
    print(f"Beta - Confidence Interval (CI):\n{CL}\n")

    # Constructing dataframe for beta coefficients
    degs = np.zeros(betas.shape[0]); degs.fill(degree)
    df = pd.DataFrame.from_dict({"degree" :degs,
                                 "coeff name": [f"b{i}" for i in range(1,betas.shape[0]+1)],
                                 "coeff value": np.round(betas, decimals=4),
                                 "Std Error": np.round(SE_betas, decimals=4),
                                 "CI lower":np.round(CI_lower_all_betas, decimals=4), 
                                 "CI_upper":np.round(CI_upper_all_betas, decimals=4)},
                                 orient='index').T
    coeffs_df = pd.concat([coeffs_df,df], axis=0)
        
    # Filling up dataframes for train and test evaluation
    z_train_OLS[degree] = z_train.flatten() 
    z_hat_train_OLS[degree] = z_hat_train.flatten()
    z_test_OLS[degree] = z_test.flatten()
    z_hat_test_OLS[degree] = z_hat_test.flatten()

    z_hat_train_SVD[degree] = z_hat_train_svd.flatten()
    z_hat_test_SVD[degree] = z_hat_test_svd.flatten()


# MSE calculations for all lambda values
mse_scores_train = ((z_train_OLS - z_hat_train_OLS) ** 2).mean()
mse_scores_test = ((z_test_OLS - z_hat_test_OLS) ** 2).mean()

mse_scores_train_svd = ((z_train_OLS - z_hat_train_SVD) ** 2).mean()
mse_scores_test_svd = ((z_test_OLS - z_hat_test_SVD) ** 2).mean()
# R2 calculations for all lambda values
#R2_scores_train = 1 - ((z_train_OLS - z_hat_train_OLS) ** 2).sum() / ((z_train_OLS - z_train_OLS.mean())**2).sum() 
#R2_scores_test = 1 - ((z_test_OLS - z_hat_test_OLS) ** 2).sum() / ((z_test_OLS - z_test_OLS.mean())**2).sum()

In [ ]:
# Plots
plt.figure(figsize=(12,8))
plt.plot(np.arange(1,degrees+1), mse_scores_train, c="c", label="Training data")
plt.plot(np.arange(1,degrees+1), mse_scores_test, c="m", label="Test data")
plt.plot(np.arange(1,degrees+1), mse_scores_train_svd, "b--", label="Training data SVD")
plt.plot(np.arange(1,degrees+1), mse_scores_test_svd, "g--", label="Test data SVD")
plt.plot(np.arange(1,degrees+1), mse_scores_train_sk, "r--", label="Training data sklearn")
plt.plot(np.arange(1,degrees+1), mse_scores_test_sk, "y--", label="Test data sklearn")
plt.xlabel("Model complexity / Polynomial Degree")
plt.ylabel("Prediction Error - MSE")
plt.title("Training evaluation on OLS regression fit given scaled data w.o. intercept")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(f"{REPORT_FIGURES}franke_function_OLS_evaluate_fit.pdf")
plt.show()

# test of summary functnion


In [ ]:
optimal_degree = 2
X = create_X(x, y, optimal_degree) # Design Matrix
model = OLS(optimal_degree) # The model
z_hat_train = model.fit(X, z.ravel()) # Fitting the model & predict on train data
summary = model.summary()
display(summary)

In [ ]:
optimal_lambda = -1
X = create_X(x, y, 3) # Design Matrix
model = RidgeRegression(optimal_lambda) # The model
z_hat_train = model.fit(X, z.ravel()) # Fitting the model & predict on train data
summary = model.summary()
display(summary)